# Extract NEs from TEIs

The notebook used here comes from the workshop "Information Extraction aus frühneuhochdeutschen Texten" (https://informationsmodellierung.uni-graz.at/de/neuigkeiten/detail/article/workshop-information-extraction-aus-fruehneuhochdeutschen-texten/). It was modified and adapted for this project.

In [1]:
from spacytei.tei import TeiReader

In [2]:
file = '../data/traindata/goldstandard.xml'# path to you file

In [3]:
teidoc = TeiReader(file)

### map your tei encoding to NE-tags

In [4]:
NER_TAG_MAP = {    
    "persName": "PER",
    "placeName": "LOC",
}

### define the tags you used for NEs via xpath
* be aware that those xpaths are relativ to a parent node (defaults to tei:p)

In [5]:
ne_xpath = './/tei:persName | .//tei:placeName'

In [6]:
ner_samples = teidoc.extract_ne_offsets(ne_xpath=ne_xpath, NER_TAG_MAP=NER_TAG_MAP)

In [7]:
ner_samples[:5]

[('Daß unser GOTT ein großer Gott ist erlernen wir unter anderen aus dem Buche der Natur der vernünftigen Heiden Bibel. Dann wenn man die zwei großen Blätter als Himmel und Erden anschauet erscheinet des Schöpfers Majestät und Herrlicheit nicht alleine aus den Wunder Lichtern des Firmaments so stets in ihrer ordentlichen Bewegung und Wirkung gehen sondern auch aus den mancherlei Gütern und Gaben so Gott über den ganzen Kreis der Erden reichlich ausgestreut. Daß man in Betrachtung dessen billig mit dem Könige David sich verwundern und ausrufen mag: Herr wie sind deine Werke so groß und viel ! du hast sie alle weißlich geordnet und die Erde ist voll deiner Güter. Weil es aber heisst: Omnia sunt creata propter hominem, & hoWo Omnia sunt creata propter hominem, & homo propter Deum. propter Deum. Das alle Dinge des Menschen wegen erschaffen der Mensch aber wegen des Schöpfers ; So will Gott auch das solche Wunderwerke nicht im Verborgen bleiben sondern von den Menschen betrachtet und Er dad

## Extract NEs from TEIs (with sent-splitting)
* The samples above are by paragraph. In case of long(er) paragraphs, you could crate NE samples splitted by sents
* Sent splitting is done by a spacy model (default 'de_core_news_sm'), so make sure you have spacy and the model you'd like to use properly installed. 

In [27]:
ner_samples = teidoc.ne_offsets_by_sent(ne_xpath=ne_xpath, NER_TAG_MAP=NER_TAG_MAP)

In case the cell above threw an error, complaining about not finding a model, try to
* install the german model `!python -m spacy download de`
* and pass in the model name to `teidoc.ne_offsets_by_sent(ne_xpath=ne_xpath, NER_TAG_MAP=NER_TAG_MAP, model='de')`

In [28]:
#!python -m spacy download de

In [29]:
#ner_samples = teidoc.ne_offsets_by_sent(ne_xpath=ne_xpath, NER_TAG_MAP=NER_TAG_MAP, model='de')

In [30]:
ner_samples[:100]

[('Daß unser GOTT ein großer Gott ist erlernen wir unter anderen aus dem Buche der Natur der vernünftigen Heiden Bibel.',
  {'entities': []}),
 ('Dann wenn man die zwei großen Blätter als Himmel und Erden anschauet erscheinet des Schöpfers Majestät und Herrlicheit nicht alleine aus den Wunder Lichtern des Firmaments so stets in ihrer ordentlichen Bewegung und Wirkung gehen sondern auch aus den mancherlei Gütern und Gaben so Gott über den ganzen Kreis der Erden reichlich ausgestreut.',
  {'entities': []}),
 ('Daß man in Betrachtung dessen billig mit dem Könige David sich verwundern und ausrufen mag:',
  {'entities': [(52, 57, 'PER')]}),
 ('Herr wie sind deine Werke so groß und viel !', {'entities': []}),
 ('du hast sie alle weißlich geordnet und die Erde ist voll deiner Güter.',
  {'entities': []}),
 ('Weil es aber heisst:', {'entities': []}),
 ('Omnia sunt creata', {'entities': []}),
 ('propter hominem, & hoWo Omnia', {'entities': []}),
 ('sunt creata', {'entities': []}),
 ('propter ho

## Extract NEs from TEIs in bulk
and save results to file

In [31]:
import os
import glob

In [32]:
from spacytei.tei_process import teis_to_traindata, teis_to_traindata_sents

In [33]:
tei_dir = '../data/traindata' # define path to directory containing TEI's

In [34]:
files = glob.glob("{}/*.xml".format(tei_dir)) # store list of relative file names of TEI's

In [35]:
samples = teis_to_traindata(files, parent_node='.//tei:body//tei:p', ne_xpath=ne_xpath, NER_TAG_MAP=NER_TAG_MAP)

In [36]:
print(samples)

[('Daß unser GOTT ein großer Gott ist erlernen wir unter anderen aus dem Buche der Natur der vernünftigen Heiden Bibel. Dann wenn man die zwei großen Blätter als Himmel und Erden anschauet erscheinet des Schöpfers Majestät und Herrlicheit nicht alleine aus den Wunder Lichtern des Firmaments so stets in ihrer ordentlichen Bewegung und Wirkung gehen sondern auch aus den mancherlei Gütern und Gaben so Gott über den ganzen Kreis der Erden reichlich ausgestreut. Daß man in Betrachtung dessen billig mit dem Könige David sich verwundern und ausrufen mag: Herr wie sind deine Werke so groß und viel ! du hast sie alle weißlich geordnet und die Erde ist voll deiner Güter. Weil es aber heisst: Omnia sunt creata propter hominem, & hoWo Omnia sunt creata propter hominem, & homo propter Deum. propter Deum. Das alle Dinge des Menschen wegen erschaffen der Mensch aber wegen des Schöpfers ; So will Gott auch das solche Wunderwerke nicht im Verborgen bleiben sondern von den Menschen betrachtet und Er dad

In [37]:
import pandas as pd

In [38]:
df = pd.DataFrame(samples, columns=['text', 'entities'])

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 2 columns):
text        534 non-null object
entities    534 non-null object
dtypes: object(2)
memory usage: 8.4+ KB


In [40]:
df.to_csv('output_csv/samples_out.csv', index=False)

In [41]:
samples = teis_to_traindata_sents(files, parent_node='.//tei:body//tei:p', ne_xpath=ne_xpath, NER_TAG_MAP=NER_TAG_MAP)

In [42]:
print(samples)

[('Daß unser GOTT ein großer Gott ist erlernen wir unter anderen aus dem Buche der Natur der vernünftigen Heiden Bibel.', {'entities': []}), ('Dann wenn man die zwei großen Blätter als Himmel und Erden anschauet erscheinet des Schöpfers Majestät und Herrlicheit nicht alleine aus den Wunder Lichtern des Firmaments so stets in ihrer ordentlichen Bewegung und Wirkung gehen sondern auch aus den mancherlei Gütern und Gaben so Gott über den ganzen Kreis der Erden reichlich ausgestreut.', {'entities': []}), ('Daß man in Betrachtung dessen billig mit dem Könige David sich verwundern und ausrufen mag:', {'entities': [(52, 57, 'PER')]}), ('Herr wie sind deine Werke so groß und viel !', {'entities': []}), ('du hast sie alle weißlich geordnet und die Erde ist voll deiner Güter.', {'entities': []}), ('Weil es aber heisst:', {'entities': []}), ('Omnia sunt creata', {'entities': []}), ('propter hominem, & hoWo Omnia', {'entities': []}), ('sunt creata', {'entities': []}), ('propter hominem, & homo pro

In [ ]:
df = pd.DataFrame(samples)
df.info()
df.to_csv('out/samples_out_sents.csv', index=False)